<a href="https://colab.research.google.com/github/chriskyfung/pandas_stock_notebook/blob/main/pandas_hkstock_datareader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Clone the entire repo.
!git clone -l -s https://github.com/chriskyfung/pandas_stock_notebook.git
%cd pandas_stock_notebook
!ls

fatal: destination path 'pandas_stock_notebook' already exists and is not an empty directory.
/content/pandas_stock_notebook
pandas_hkstock_datareader.ipynb  requirements.txt  stocks.py
README.md			 stocks.json


In [4]:
!pip install -r requirements.txt

In [10]:
from datetime import date, timedelta
import pandas as pd
import pandas_market_calendars as mcal

# Create a calendar
hkex = mcal.get_calendar('HKEX')

today = date.today()
past7days = today - timedelta(days=7)
print("Today's date:", today)

# Show available calendars in the past 7 days
open_dates = hkex.valid_days(past7days,today)
print(f'\nOpen dates (past 7 days):\n {open_dates.date}')

# Show the last stock open date
last_open_date = open_dates[-1].date()
print(f'\nLast open date: {last_open_date}')

Today's date: 2021-06-30

Open dates (past 7 days):
 [datetime.date(2021, 6, 23) datetime.date(2021, 6, 24)
 datetime.date(2021, 6, 25) datetime.date(2021, 6, 28)
 datetime.date(2021, 6, 29) datetime.date(2021, 6, 30)]

Last open date: 2021-06-30


In [11]:
from stocks import stock_profile # 從 stocks.py 載入 stock_profile 類別
  
stocks = stock_profile()
stocks.loadJsonProfile('./stocks.json') # 從JSON文件讀入股票組合清單及股票名稱索引
stocks.cacheFromYahooData(stocks.listing, last_open_date) # 從Yahoo下載股票組合之最近一天資料


騰訊(0700.hk):


,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2021-06-30,597.5,584,597.5,584,13618352,584




中石油(0857.hk):


,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2021-06-30,3.79,3.67,3.67,3.78,186178750,3.78




中海油(0883.hk):


,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2021-06-30,9.03,8.81,8.88,8.83,68328296,8.83




小米(1810.hk):


,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2021-06-30,27.65,26.85,27.5,27,72207758,27


In [12]:
from termcolor import colored
import re

asked_stock = input('Enter a stock code/name: ') # 詢問要查閱的股票代號/名稱

stock_code = None
stock_name = None
# 判辨輸入的字串是股票代號還是股票名稱
if re.match('\d+\.\w+', asked_stock):
  stock_code = asked_stock.lower()
  stock_name = stocks.code2name(stock_code)
else:
  stock_code = stocks.name2code(asked_stock)
  stock_name = asked_stock
# 從快取要查閱的股票資料
if (stock_code in stocks.dataframes):
  df = stocks.dataframes[stock_code]
  columns = list(df) # or df.columns
  # 將最近一天的資料Print到屏幕輸出
  print(f'\nStock data of {stock_name}({stock_code}) on the last open day ({last_open_date})\n')
  for column in columns:
    print(f'{column:9s}: {df.iloc[-1].loc[column]:>15,.2f}')
else:
  # 如果沒有快取，則輸出紅色的文字提示！
  print(colored(f'\nCannot find "{asked_stock}" from the cached data', 'red'))

Enter a stock code/name: 0883.hk

Stock data of 中海油(0883.hk) on the last open day (2021-06-30)

High     :            9.03
Low      :            8.81
Open     :            8.88
Close    :            8.83
Volume   :   68,328,296.00
Adj Close:            8.83
